## Import necessary libraries

In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from scipy.stats import chi2_contingency
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score


## Dataset Loading

In [61]:
train_data = pd.read_csv('/home/sachin/DSP/dsp-anandhu-krishna/data/train.csv')
test_data = pd.read_csv('/home/sachin/DSP/dsp-anandhu-krishna/data/test.csv')

In [62]:
#storing test_data_ids into a list
test_data_id= list(test_data['Id'])

In [63]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [64]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Data cleaning 

In [65]:
#finding missing value in train data 
missing_values = train_data.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print("Columns with missing values and their counts in train data:")
print(missing_columns)

#finding missing value in test data 
missing_values = test_data.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print("Columns with missing values and their counts in test data:")
print(missing_columns)


Columns with missing values and their counts in train data:
LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64
Columns with missing values and their counts in test data:
MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType       894
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageT

In [66]:
#dropping columns with higehst missing value in train and test data
columns_to_drop = ['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature','FireplaceQu','Id']
train_data.drop(columns_to_drop, axis=1, inplace=True)
test_data.drop(columns_to_drop, axis=1, inplace=True)


In [67]:
#finding highest correaled columns 

numerical_train_data = train_data.select_dtypes(include=[np.number])

correlation_matrix = numerical_train_data.corr()

# Initialize a list to hold the pairs with high correlation
high_correlation_pairs = []

# Iterate over the correlation matrix
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)): # i+1 to avoid self-comparison
        if abs(correlation_matrix.iloc[i, j]) > 0.75: # Check for absolute correlation > 0.75
            high_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

# Convert the list of high correlation pairs into a DataFrame for better readability
high_corr_df = pd.DataFrame(high_correlation_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

# Display the filtered pairs
print(high_corr_df)

     Feature 1     Feature 2  Correlation
0  OverallQual     SalePrice     0.790982
1    YearBuilt   GarageYrBlt     0.825667
2  TotalBsmtSF      1stFlrSF     0.819530
3    GrLivArea  TotRmsAbvGrd     0.825489
4   GarageCars    GarageArea     0.882475


In [68]:
#From this correaltion we can drop columns GarageYrBlt,1stFlrSF,TotRmsAbvGrd,GarageArea Since 2 columns(other than target variable ) have high correlation

train_data.drop(['GarageYrBlt','1stFlrSF','TotRmsAbvGrd','GarageArea'], axis=1, inplace=True)
test_data.drop(['GarageYrBlt','1stFlrSF','TotRmsAbvGrd','GarageArea'], axis=1, inplace=True)

In [69]:
#finding missing value in train data 
missing_values = train_data.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print("Columns with missing values and their counts in train data:")
print(missing_columns)

Columns with missing values and their counts in train data:
LotFrontage     259
MasVnrArea        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
GarageType       81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64


In [70]:
#filling missing values in  train data for  both numerical and categorical features with mean value and mode value respectively

numerical_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_data.select_dtypes(exclude=[np.number]).columns.tolist()

# Fill missing values for numerical columns with the mean
for col in numerical_cols:
    train_data[col] = train_data[col].fillna(train_data[col].mean())

# Fill missing values for categorical columns with the mode
for col in categorical_cols:
    train_data[col] = train_data[col].fillna(train_data[col].mode()[0])

In [71]:
#filling missing values in  test data for  both numerical and categorical features with mean value and mode value respectively

numerical_cols = test_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = test_data.select_dtypes(exclude=[np.number]).columns.tolist()

# Fill missing values for numerical columns with the mean
for col in numerical_cols:
    test_data[col] = test_data[col].fillna(test_data[col].mean())

# Fill missing values for categorical columns with the mode
for col in categorical_cols:
    test_data[col] = test_data[col].fillna(test_data[col].mode()[0])

In [72]:
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


## Feature selection

In [73]:
#finding the correlation between numerical columns and SalePrice
numerical_train_data = train_data.select_dtypes(include=[np.number])
numerical_correlations = numerical_train_data.corr()['SalePrice'].abs().sort_values(ascending=False)

#taking the top 10 correlated features
top_10_correlated_numerical_features = numerical_correlations[1:11]


# Print the top_10_correlated_numerical_features
print("Numerical Features with highest Correlation:")
print(top_10_correlated_numerical_features)

Numerical Features with highest Correlation:
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
TotalBsmtSF     0.613581
FullBath        0.560664
YearBuilt       0.522897
YearRemodAdd    0.507101
MasVnrArea      0.475241
Fireplaces      0.466929
BsmtFinSF1      0.386420
Name: SalePrice, dtype: float64


In [74]:
#selecting the categorical features
categorical_features = [column for column in train_data.columns if train_data[column].dtype == 'object']

# Perform chi-square test for categorical features
chi2_results = {}
for feature in categorical_features:
    contingency_table = pd.crosstab(train_data[feature], train_data['SalePrice'])
    chi2_statistic, _, _, _ = chi2_contingency(contingency_table)
    chi2_results[feature] = chi2_statistic

# selecting top 5 chi-square test results for categorical features
top_5_categorical_features = dict(sorted(chi2_results.items(), key=lambda x: x[1], reverse=True)[:5])


# Print top 5 chi-square test results for categorical features
print("\nTop 5 Chi-square Test Results for Categorical Features:")
for feature, result in top_5_categorical_features.items():
    print(f"{feature}: {result}")



Top 5 Chi-square Test Results for Categorical Features:
Neighborhood: 16898.75578956907
Exterior2nd: 9785.82598370394
Exterior1st: 8712.732778251357
SaleType: 6099.7944527529235
Condition2: 4772.610202894631


In [75]:
# Convert top 10 correlated numerical features to a list
top_10_numerical_features = top_10_correlated_numerical_features.index.tolist()
# Convert top 5 correlated categorical features to a list
top_5_categorical_features_list = list(top_5_categorical_features.keys())

# Concatenate top correlated numerical features and top chi-square test results for categorical features to get the final features
final_features = top_10_numerical_features + top_5_categorical_features_list

#adding target variable(SalePrice) to features list 
final_features.append('SalePrice')
final_features

['OverallQual',
 'GrLivArea',
 'GarageCars',
 'TotalBsmtSF',
 'FullBath',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'Fireplaces',
 'BsmtFinSF1',
 'Neighborhood',
 'Exterior2nd',
 'Exterior1st',
 'SaleType',
 'Condition2',
 'SalePrice']

## Train test split

In [76]:
X = train_data[final_features].drop(columns=['SalePrice'])  # Drop the target column from features
y = train_data['SalePrice']  # Target column

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature Processing

In [77]:
# Feature Processing
numeric_features = top_10_numerical_features

categorical_features = top_5_categorical_features_list

# Numeric feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])
test_data_scaled = scaler.transform(test_data[numeric_features])


# Categorical feature encoding
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_features])
X_test_encoded = encoder.transform(X_test[categorical_features])
test_data_encoded = encoder.transform(test_data[categorical_features])


# Combine scaled numeric features and encoded categorical features
X_train_processed = np.hstack((X_train_scaled, X_train_encoded.toarray()))
X_test_processed = np.hstack((X_test_scaled, X_test_encoded.toarray()))
test_data_processed = np.hstack((test_data_scaled, test_data_encoded.toarray()))


## Model Training

In [78]:

#used xgboost algorithm for training the model
param = {
    'max_depth': 4,            
    'objective': 'reg:squarederror',  
    'learning_rate': 0.1,   
    'n_estimators': 200,       
    'subsample': 0.7,          
    'colsample_bytree': 0.8,    
    'eval_metric': 'rmse'       
}


model = XGBRegressor(**param)

#model training
model.fit(X_train_processed, y_train)
#model predicitng
y_pred = model.predict(X_test_processed)


## Evaluating the model 

In [79]:
# function for calucalting Root Mean Squared Logarithmic Error

def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

In [80]:
rmsle_score = compute_rmsle(np.log(y_test), np.log(y_pred))
print("Root Mean Squared Logarithmic Error (RMSLE):", rmsle_score)

Root Mean Squared Logarithmic Error (RMSLE): 0.01


In [81]:
#function for calculating the other metrices
def evaluate_regression_performance(y_test, y_pred):
    mse = np.mean((y_test - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    r2 = r2_score(y_test, y_pred)
    return mse, rmse, mae, r2


In [83]:
mse, rmse, mae, r2=evaluate_regression_performance(y_test, y_pred) 
print(" MSE is ", mse)
print(" RMSE is ", rmse)
print(" MAE is ", mae)
print("R^2 score is ", r2)



 MSE is  726993141.131354
 RMSE is  26962.81033444685
 MAE is  17477.019330586474
R^2 score is  0.9052200491225316


## predicitng on test data

In [84]:
test_data_pred = model.predict(test_data_processed)

In [85]:
#storing the predicted slaes price into a df

test_data_pred_df = pd.DataFrame(test_data_pred, columns=['pred_sales_price'])

#converting the Ids into a dataframe
test_data_id_df = pd.DataFrame(test_data_id, columns=['id'])

# Concatenate the new DataFrame with 'test_data_id_df' 
df_combined = pd.concat([test_data_id_df, test_data_pred_df], axis=1)

In [86]:
#printing the predicted salesprice with Ids

df_combined

,id,pred_sales_price
0,1461,120539.046875
1,1462,144140.406250
2,1463,179354.078125
3,1464,187382.281250
4,1465,200444.000000
...,...,...
1454,2915,70883.984375
1455,2916,84315.437500
1456,2917,166778.421875
1457,2918,111551.804688
